In [1]:
# Import the libraries
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style

import tensorflow as tf
from tensorflow import keras
from keras import metrics
tf.get_logger().setLevel('INFO')

from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
#load data
X_train = pd.read_csv('../data/processed/X_train.csv')
Y_train = pd.read_csv('../data/processed/Y_train.csv')
X_test = pd.read_csv('../data/processed/X_test.csv')
Y_test = pd.read_csv('../data/processed/Y_test.csv')

### Implementing Baseline

In [3]:
#show class distribution
class_counts = Y_train.value_counts().reset_index()
class_counts

,music_category,count
0,1,9318
1,0,5701
2,4,5534
3,2,4456
4,3,2689
5,8,2214
6,5,1826
7,7,1341
8,6,1129


Since class distribution isn't too unbalanced, implement majority baseline and stratified baseline. ## EDIT class majority big now

In [4]:
#majority baseline implementation
dummy_clf = DummyClassifier(strategy = 'stratified', random_state = 42)
dummy_clf.fit(X_train, Y_train)
Y_pred = dummy_clf.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test, Y_pred)

print(f'Baseline Classifier Accuracy: {accuracy}')
print(f'Classification Report: {report}')

Baseline Classifier Accuracy: 0.1590085350169531
Classification Report:               precision    recall  f1-score   support

           0       0.15      0.15      0.15      1339
           1       0.28      0.28      0.28      2422
           2       0.12      0.11      0.12      1179
           3       0.08      0.09      0.09       651
           4       0.14      0.15      0.15      1306
           5       0.06      0.06      0.06       454
           6       0.05      0.04      0.04       304
           7       0.03      0.03      0.03       312
           8       0.07      0.07      0.07       586

    accuracy                           0.16      8553
   macro avg       0.11      0.11      0.11      8553
weighted avg       0.16      0.16      0.16      8553



#### Testing for uniform distribution of classes

In [5]:
min_count = class_counts['count'].min()
result = {}
unique_classes = np.unique(Y_train)
for value in unique_classes:
    indices = np.where(Y_train==value)[0][:min_count]
    result[value] = indices.tolist()
index_values = list(result.values())
index_values = [element for nestedlist in index_values for element in nestedlist]
X_train_uniform = X_train.iloc[index_values]
Y_train_uniform = Y_train.iloc[index_values]

In [6]:
X_train_uniform.info(20)

<class 'pandas.core.frame.DataFrame'>
Index: 10161 entries, 0 to 17232
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      10161 non-null  float64
 1   energy            10161 non-null  float64
 2   loudness          10161 non-null  float64
 3   mode              10161 non-null  int64  
 4   speechiness       10161 non-null  float64
 5   acousticness      10161 non-null  float64
 6   instrumentalness  10161 non-null  float64
 7   liveness          10161 non-null  float64
 8   valence           10161 non-null  float64
 9   tempo             10161 non-null  float64
 10  C                 10161 non-null  int64  
 11  C#                10161 non-null  int64  
 12  D                 10161 non-null  int64  
 13  D#                10161 non-null  int64  
 14  E                 10161 non-null  int64  
 15  F                 10161 non-null  int64  
 16  F#                10161 non-null  int64  
 17

### Building a Model

In [7]:
len(Y_train['music_category'].unique())

9

In [8]:
X_train.shape

(34208, 22)

In [9]:
def build_model(num_features, n_classes,learning_rate=0.01):
    """Build a TF logistic regression model using Keras.
    
    Args:
    learning_rate: The desired learning rate for SGD.
    
    Returns:
    model: A tf.keras model (graph).
    """
    # Random Seed + Clear Session
    tf.keras.backend.clear_session()
    np.random.seed(0)
    tf.random.set_seed(0)
    
    # Build a model using keras.Sequential.
    model = keras.Sequential(name = 'Genres')
    
    # Keras layers processing
    model.add(tf.keras.layers.InputLayer(input_shape=num_features))
    
    # This layer constructs the linear set of parameters for each input feature
    # (as well as a bias), and applies a sigmoid to the result. The result is
    # binary logistic regression.
    model.add(keras.layers.Dense(
      units=n_classes,
      activation='softmax'))

    # Use the SGD optimizer as usual.
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    
    # We specify the binary_crossentropy loss (equivalent to log loss).
    # Notice that we are including 'binary accuracy' as one of the metrics that we
    # ask Tensorflow to report when evaluating the model.
    model.compile(loss='sparse_categorical_crossentropy', 
                optimizer=optimizer, 
                metrics=['accuracy'])
    
    return model

In [10]:
model = build_model(num_features = X_train.shape[1], n_classes = len(Y_train['music_category'].unique()))
model.summary()


Model: "Genres"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 207       
                                                                 
Total params: 207 (828.00 Byte)
Trainable params: 207 (828.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
# Fit the model.
history = model.fit(
  x = X_train,   # training examples
  y = Y_train,   #labels
  epochs=5,             # number of passes through the training data
  batch_size=64,        # mini-batch size for SGD
  validation_split=0.1, # use a fraction of the examples for validation -- DO WE STILL NEED THIS IF VALIDATION SPLIT
  verbose=1             # display some progress output during training
  )

# Convert the return value into a DataFrame so we can see the train loss 
# and binary accuracy after every epoch.
history = pd.DataFrame(history.history)
display(history)

Epoch 1/5


482/482 [==============================] - 1s 988us/step - loss: 61.3400 - accuracy: 0.1637 - val_loss: 102.1034 - val_accuracy: 0.1751
Epoch 2/5
482/482 [==============================] - 0s 773us/step - loss: 59.9687 - accuracy: 0.1689 - val_loss: 60.7173 - val_accuracy: 0.1339
Epoch 3/5
482/482 [==============================] - 0s 748us/step - loss: 57.4975 - accuracy: 0.1744 - val_loss: 80.5075 - val_accuracy: 0.2789
Epoch 4/5
482/482 [==============================] - 0s 757us/step - loss: 55.9011 - accuracy: 0.1761 - val_loss: 66.4973 - val_accuracy: 0.1599
Epoch 5/5
482/482 [==============================] - 0s 761us/step - loss: 56.0176 - accuracy: 0.1789 - val_loss: 137.3372 - val_accuracy: 0.0742


,loss,accuracy,val_loss,val_accuracy
0,61.339958,0.163673,102.103363,0.175095
1,59.968723,0.168870,60.717285,0.133879
2,57.497471,0.174424,80.507538,0.278866
3,55.901131,0.176113,66.497276,0.159895
4,56.017597,0.178874,137.337234,0.074247


##### TESTING for EVENLY distributed classes

In [12]:
model = build_model(num_features = X_train_uniform.shape[1], n_classes = len(Y_train['music_category'].unique()))
model.summary()

Model: "Genres"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 207       
                                                                 
Total params: 207 (828.00 Byte)
Trainable params: 207 (828.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Fit the model.
history = model.fit(
  x = X_train_uniform,   # training examples
  y = Y_train_uniform,   #labels
  epochs=5,             # number of passes through the training data
  batch_size=64,        # mini-batch size for SGD
  validation_split=0.1, # use a fraction of the examples for validation -- DO WE STILL NEED THIS IF VALIDATION SPLIT
  verbose=1             # display some progress output during training
  )

# Convert the return value into a DataFrame so we can see the train loss 
# and binary accuracy after every epoch.
history = pd.DataFrame(history.history)
display(history)

Epoch 1/5
143/143 [==============================] - 0s 1ms/step - loss: 60.8676 - accuracy: 0.1226 - val_loss: 151.1744 - val_accuracy: 0.0000e+00
Epoch 2/5
143/143 [==============================] - 0s 845us/step - loss: 57.5296 - accuracy: 0.1517 - val_loss: 136.0601 - val_accuracy: 0.0000e+00
Epoch 3/5
143/143 [==============================] - 0s 845us/step - loss: 53.6362 - accuracy: 0.1652 - val_loss: 148.0462 - val_accuracy: 0.0000e+00
Epoch 4/5
143/143 [==============================] - 0s 880us/step - loss: 52.1524 - accuracy: 0.1730 - val_loss: 57.0609 - val_accuracy: 0.0000e+00
Epoch 5/5
143/143 [==============================] - 0s 852us/step - loss: 54.1028 - accuracy: 0.1638 - val_loss: 53.9247 - val_accuracy: 0.0000e+00


,loss,accuracy,val_loss,val_accuracy
0,60.867565,0.122594,151.174362,0.0
1,57.529568,0.151684,136.060089,0.0
2,53.636158,0.165245,148.046188,0.0
3,52.152367,0.173010,57.060940,0.0
4,54.102818,0.163823,53.924686,0.0


### Feature Scaling & Normalization

In [14]:
X_train.describe()

,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,...,D,D#,E,F,F#,G,G#,A,A#,B
count,34208.000000,34208.000000,34208.000000,34208.000000,34208.000000,34208.000000,34208.000000,34208.000000,34208.000000,34208.000000,...,34208.000000,34208.000000,34208.000000,34208.000000,34208.000000,34208.000000,34208.000000,34208.000000,34208.000000,34208.000000
mean,0.567326,0.683304,-7.654820,0.626345,0.078443,0.273082,0.171430,0.222305,0.485916,124.526902,...,0.104040,0.028707,0.081881,0.079894,0.068639,0.120674,0.059284,0.100678,0.067148,0.081648
std,0.170185,0.229939,4.045018,0.483781,0.078316,0.313001,0.315133,0.196574,0.261501,28.689902,...,0.305317,0.166983,0.274188,0.271132,0.252843,0.325752,0.236160,0.300907,0.250282,0.273831
min,0.000000,0.000020,-40.046000,0.000000,0.000000,0.000000,0.000000,0.009860,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.456000,0.525000,-9.462250,0.000000,0.035900,0.006208,0.000000,0.097600,0.269000,102.902250,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.579000,0.723000,-6.824000,1.000000,0.049100,0.119000,0.000113,0.138000,0.477000,123.979000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.695000,0.877000,-4.922000,1.000000,0.083600,0.513000,0.122000,0.295000,0.698000,142.043500,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.985000,1.000000,4.532000,1.000000,0.939000,0.996000,0.999000,1.000000,0.994000,222.605000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
X_train_features_norm = (X_train_uniform - X_train_uniform.mean())/X_train_uniform.std() #X_train
X_test_features_norm = (X_test - X_train_uniform.mean())/X_train.std() 
#only train features shown since we're not supposed to look at test
X_train_features_norm.describe()

,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,...,D,D#,E,F,F#,G,G#,A,A#,B
count,1.016100e+04,1.016100e+04,1.016100e+04,1.016100e+04,1.016100e+04,1.016100e+04,1.016100e+04,1.016100e+04,1.016100e+04,1.016100e+04,...,1.016100e+04,1.016100e+04,1.016100e+04,1.016100e+04,1.016100e+04,1.016100e+04,1.016100e+04,1.016100e+04,1.016100e+04,1.016100e+04
mean,-2.125824e-16,-1.566397e-16,-1.957996e-16,2.797137e-17,8.391411e-17,-8.950839e-17,-4.475419e-17,2.237710e-17,-8.950839e-17,1.489475e-16,...,3.076851e-17,2.657280e-17,5.314560e-17,-5.734131e-17,9.789980e-18,-8.671125e-17,-2.797137e-18,4.755133e-17,5.804059e-17,-4.615276e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-3.020039e+00,-2.459836e+00,-6.365982e+00,-1.350805e+00,-9.517743e-01,-9.934191e-01,-5.048343e-01,-1.076130e+00,-1.827960e+00,-4.153901e+00,...,-3.491170e-01,-1.916438e-01,-3.079091e-01,-3.007676e-01,-2.616248e-01,-3.738324e-01,-2.408131e-01,-3.405419e-01,-2.644049e-01,-2.889378e-01
25%,-6.686993e-01,-7.201733e-01,-3.608115e-01,-1.350805e+00,-5.255894e-01,-9.556774e-01,-5.048343e-01,-6.265948e-01,-8.359191e-01,-7.569017e-01,...,-3.491170e-01,-1.916438e-01,-3.079091e-01,-3.007676e-01,-2.616248e-01,-3.738324e-01,-2.408131e-01,-3.405419e-01,-2.644049e-01,-2.889378e-01
50%,7.092386e-02,1.452839e-01,2.367851e-01,7.402264e-01,-3.782055e-01,-3.605196e-01,-5.045281e-01,-4.268016e-01,-3.176479e-02,-3.616228e-02,...,-3.491170e-01,-1.916438e-01,-3.079091e-01,-3.007676e-01,-2.616248e-01,-3.738324e-01,-2.408131e-01,-3.405419e-01,-2.644049e-01,-2.889378e-01
75%,7.608708e-01,8.587014e-01,6.541157e-01,7.402264e-01,4.429481e-02,8.791505e-01,-3.130485e-01,3.536408e-01,8.062091e-01,6.078600e-01,...,-3.491170e-01,-1.916438e-01,-3.079091e-01,-3.007676e-01,-2.616248e-01,-3.738324e-01,-2.408131e-01,-3.405419e-01,-2.644049e-01,-2.889378e-01
max,2.416744e+00,1.431774e+00,2.066635e+00,7.402264e-01,1.008973e+01,1.898177e+00,2.789094e+00,4.042532e+00,1.907224e+00,3.412605e+00,...,2.864087e+00,5.217501e+00,3.247392e+00,3.324499e+00,3.821892e+00,2.674732e+00,4.152189e+00,2.936207e+00,3.781706e+00,3.460612e+00


### Model 2

In [16]:
def build_model2(num_features, n_classes,learning_rate=0.01):
    """Build a TF logistic regression model using Keras.
    
    Args:
    learning_rate: The desired learning rate for SGD.
    
    Returns:
    model: A tf.keras model (graph).
    """
    # Random Seed + Clear Session
    tf.keras.backend.clear_session()
    np.random.seed(0)
    tf.random.set_seed(0)
    
    # Build a model using keras.Sequential.
    model = keras.Sequential(name = 'Genres')
    
    # Keras layers processing
    model.add(tf.keras.layers.InputLayer(input_shape=num_features))

    #adding extra layer
    model.add(keras.layers.Dense(
            units=256,
            activation = 'relu'))

    #add extra layer
    model.add(keras.layers.Dense(
            units=128,
            activation = 'relu'))
    
    # This layer constructs the linear set of parameters for each input feature
    # (as well as a bias), and applies a sigmoid to the result. The result is
    # binary logistic regression.
    model.add(keras.layers.Dense(
      units=n_classes,
      activation='softmax'))

    # Use the SGD optimizer as usual.
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    
    # We specify the binary_crossentropy loss (equivalent to log loss).
    # Notice that we are including 'binary accuracy' as one of the metrics that we
    # ask Tensorflow to report when evaluating the model.
    model.compile(loss='sparse_categorical_crossentropy', 
                optimizer=optimizer, 
                metrics=['accuracy'])
    
    return model

In [17]:
model2 = build_model2(num_features = X_train_features_norm.shape[1], n_classes = len(Y_train['music_category'].unique()))
model2.summary()

Model: "Genres"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               5888      
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 9)                 1161      
                                                                 
Total params: 39945 (156.04 KB)
Trainable params: 39945 (156.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
# Fit the model.
history2 = model2.fit(
  x = X_train_features_norm,   # training examples
  y = Y_train,   #labels
  epochs=10,             # number of passes through the training data
  batch_size=128,        # mini-batch size for SGD
  validation_split=0.1, # use a fraction of the examples for validation -- DO WE STILL NEED THIS IF VALIDATION SPLIT
  verbose=1             # display some progress output during training
  )

# Convert the return value into a DataFrame so we can see the train loss 
# and binary accuracy after every epoch.
history2 = pd.DataFrame(history2.history)
display(history2)

Epoch 1/10
72/72 [==============================] - 0s 3ms/step - loss: 2.1328 - accuracy: 0.2247 - val_loss: 2.0964 - val_accuracy: 0.2340
Epoch 2/10
72/72 [==============================] - 0s 1ms/step - loss: 2.0323 - accuracy: 0.2796 - val_loss: 2.0569 - val_accuracy: 0.2360
Epoch 3/10
72/72 [==============================] - 0s 1ms/step - loss: 2.0074 - accuracy: 0.2813 - val_loss: 2.0436 - val_accuracy: 0.2389
Epoch 4/10
72/72 [==============================] - 0s 1ms/step - loss: 1.9977 - accuracy: 0.2827 - val_loss: 2.0372 - val_accuracy: 0.2389
Epoch 5/10
72/72 [==============================] - 0s 1ms/step - loss: 1.9920 - accuracy: 0.2841 - val_loss: 2.0341 - val_accuracy: 0.2439
Epoch 6/10
72/72 [==============================] - 0s 1ms/step - loss: 1.9880 - accuracy: 0.2844 - val_loss: 2.0316 - val_accuracy: 0.2448
Epoch 7/10
72/72 [==============================] - 0s 1ms/step - loss: 1.9849 - accuracy: 0.2844 - val_loss: 2.0308 - val_accuracy: 0.2439
Epoch 8/10
72/72 [==

,loss,accuracy,val_loss,val_accuracy
0,2.132761,0.224738,2.096397,0.234022
1,2.032274,0.279637,2.056947,0.235988
2,2.007376,0.281277,2.043590,0.238938
3,1.997655,0.282699,2.037169,0.238938
4,1.992026,0.284121,2.034085,0.243854
5,1.987998,0.284449,2.031603,0.244838
6,1.984925,0.284449,2.030768,0.243854
7,1.982253,0.285542,2.028916,0.245821
8,1.979958,0.286199,2.028777,0.246804
9,1.977987,0.286089,2.028025,0.246804


### Testing - Feature Selection

In [19]:
X_train_features_norm.columns

Index(['danceability', 'energy', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'C',
       'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'],
      dtype='object')

In [20]:
#features = ['danceability', 'energy', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
features = ['danceability','energy', 'loudness', 'valence', 'tempo']
X_train_features_norm = X_train_features_norm[features]

In [21]:
X_train_features_norm.head()

,danceability,energy,loudness,valence,tempo
0,-0.122261,1.158882,-0.024248,0.261338,1.403685
1,-0.850845,1.018538,0.299022,1.012884,1.159733
2,0.462814,-0.244562,0.500235,1.456296,-1.737908
4,0.208913,-0.727970,-0.221836,0.437952,0.599362
6,-0.712856,1.162781,1.252719,-0.009218,2.737005


In [22]:
model2 = build_model2(num_features = len(features), n_classes = len(Y_train['music_category'].unique()))
model2.summary()

Model: "Genres"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               1536      
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 9)                 1161      
                                                                 
Total params: 35593 (139.04 KB)
Trainable params: 35593 (139.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
# Fit the model.
history2 = model2.fit(
  x = X_train_features_norm,   # training examples
  y = Y_train_uniform,   #labels
  epochs=10,             # number of passes through the training data
  batch_size=128,        # mini-batch size for SGD
  validation_split=0.1, # use a fraction of the examples for validation -- DO WE STILL NEED THIS IF VALIDATION SPLIT
  verbose=1             # display some progress output during training
  )

# Convert the return value into a DataFrame so we can see the train loss 
# and binary accuracy after every epoch.
history2 = pd.DataFrame(history2.history)
display(history2)

Epoch 1/10
72/72 [==============================] - 0s 3ms/step - loss: 2.1090 - accuracy: 0.2207 - val_loss: 2.5707 - val_accuracy: 0.0000e+00
Epoch 2/10
72/72 [==============================] - 0s 1ms/step - loss: 1.9922 - accuracy: 0.3082 - val_loss: 2.7716 - val_accuracy: 0.0000e+00
Epoch 3/10
72/72 [==============================] - 0s 1ms/step - loss: 1.9051 - accuracy: 0.3444 - val_loss: 2.9520 - val_accuracy: 0.0000e+00
Epoch 4/10
72/72 [==============================] - 0s 1ms/step - loss: 1.8376 - accuracy: 0.3650 - val_loss: 3.1126 - val_accuracy: 0.0000e+00
Epoch 5/10
72/72 [==============================] - 0s 1ms/step - loss: 1.7833 - accuracy: 0.3743 - val_loss: 3.2578 - val_accuracy: 0.0000e+00
Epoch 6/10
72/72 [==============================] - 0s 1ms/step - loss: 1.7396 - accuracy: 0.3876 - val_loss: 3.3905 - val_accuracy: 0.0000e+00
Epoch 7/10
72/72 [==============================] - 0s 1ms/step - loss: 1.7048 - accuracy: 0.3951 - val_loss: 3.5088 - val_accuracy: 0.0

,loss,accuracy,val_loss,val_accuracy
0,2.109006,0.220691,2.570744,0.0
1,1.992230,0.308180,2.771585,0.0
2,1.905050,0.344379,2.951958,0.0
3,1.837593,0.365048,3.112595,0.0
4,1.783288,0.374344,3.257784,0.0
5,1.739557,0.387577,3.390496,0.0
6,1.704810,0.395122,3.508797,0.0
7,1.676870,0.401137,3.614108,0.0
8,1.654193,0.403653,3.706316,0.0
9,1.635622,0.409230,3.788519,0.0


In [24]:
X_test_features_norm = X_test_features_norm[features]
model2.predict(X_test_features_norm)

268/268 [==============================] - 0s 667us/step


array([[0.10301549, 0.12869109, 0.14284793, ..., 0.0694591 , 0.15134056,
        0.02370277],
       [0.03824765, 0.06306092, 0.10426366, ..., 0.00601698, 0.3195092 ,
        0.00732152],
       [0.11734904, 0.06341486, 0.15252203, ..., 0.02610851, 0.17651589,
        0.02464006],
       ...,
       [0.05922796, 0.12006798, 0.13107538, ..., 0.01537286, 0.24464266,
        0.01495103],
       [0.10005479, 0.21090555, 0.1403931 , ..., 0.05425014, 0.12504478,
        0.03279598],
       [0.1101208 , 0.08093734, 0.1918542 , ..., 0.15083617, 0.12210994,
        0.02378812]], dtype=float32)

In [25]:
print("Evaluate on test data")
results = model2.evaluate(X_test_features_norm, Y_test, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
67/67 [==============================] - 0s 833us/step - loss: 1.8971 - accuracy: 0.3097
test loss, test acc: [1.8970797061920166, 0.3097158968448639]


### Model 3 Testing

In [26]:
def build_model3(num_features, n_classes,learning_rate=0.01):
    """Build a TF logistic regression model using Keras.
    
    Args:
    learning_rate: The desired learning rate for SGD.
    
    Returns:
    model: A tf.keras model (graph).
    """
    # Random Seed + Clear Session
    tf.keras.backend.clear_session()
    np.random.seed(0)
    tf.random.set_seed(0)
    
    # Build a model using keras.Sequential.
    model = keras.Sequential(name = 'Genres')
    
    # Keras layers processing
    model.add(tf.keras.layers.InputLayer(input_shape=num_features))

    model.add(tf.keras.layers.Dense(256, activation = 'relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(128, activation = "relu"))
    model.add(keras.layers.Dense(128, activation = "relu"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(64, activation = "relu"))
    model.add(keras.layers.Dense(
      units=n_classes,
      activation='softmax'))
    
    
    model.compile(loss='sparse_categorical_crossentropy', 
                optimizer= keras.optimizers.Adam(), 
                metrics=['accuracy'])
    
    return model

In [27]:
model3 = build_model3(num_features = len(features), n_classes = len(Y_train['music_category'].unique()))

In [28]:
# Fit the model.
history3 = model3.fit(
  x = X_train_features_norm,   # training examples
  y = Y_train_uniform,   #labels
  epochs=100,             # number of passes through the training data
  batch_size=128,        # mini-batch size for SGD
  validation_split=0.2, # use a fraction of the examples for validation -- DO WE STILL NEED THIS IF VALIDATION SPLIT
  verbose=1             # display some progress output during training
  )

# Convert the return value into a DataFrame so we can see the train loss 
# and binary accuracy after every epoch.
history3 = pd.DataFrame(history3.history)
display(history3)

Epoch 1/100
64/64 [==============================] - 1s 4ms/step - loss: 1.6316 - accuracy: 0.4064 - val_loss: 2.8759 - val_accuracy: 0.0000e+00
Epoch 2/100
64/64 [==============================] - 0s 2ms/step - loss: 1.4383 - accuracy: 0.4630 - val_loss: 3.6565 - val_accuracy: 0.0000e+00
Epoch 3/100
64/64 [==============================] - 0s 2ms/step - loss: 1.4081 - accuracy: 0.4694 - val_loss: 4.1336 - val_accuracy: 0.0000e+00
Epoch 4/100
64/64 [==============================] - 0s 2ms/step - loss: 1.3907 - accuracy: 0.4771 - val_loss: 4.5572 - val_accuracy: 0.0000e+00
Epoch 5/100
64/64 [==============================] - 0s 2ms/step - loss: 1.3749 - accuracy: 0.4840 - val_loss: 4.7443 - val_accuracy: 0.0000e+00
Epoch 6/100
64/64 [==============================] - 0s 2ms/step - loss: 1.3656 - accuracy: 0.4828 - val_loss: 4.9101 - val_accuracy: 0.0000e+00
Epoch 7/100
64/64 [==============================] - 0s 2ms/step - loss: 1.3580 - accuracy: 0.4860 - val_loss: 5.0012 - val_accura

,loss,accuracy,val_loss,val_accuracy
0,1.631550,0.406373,2.875945,0.000000
1,1.438271,0.462968,3.656495,0.000000
2,1.408091,0.469365,4.133595,0.000000
3,1.390691,0.477116,4.557198,0.000000
4,1.374943,0.484006,4.744341,0.000000
...,...,...,...,...
95,1.151801,0.556348,9.110151,0.028037
96,1.144401,0.558563,9.191340,0.018692
97,1.154741,0.553765,9.055189,0.019183
98,1.142357,0.562008,9.254242,0.013281
